# Jupyter Notebook to clean install all the contracts

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint

from repeater import retry

NONCE = 100
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"


@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [3]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

# Networks
Uncomment the line which you want to use.

In [4]:
# Mainnet
# icon_service = IconService(HTTPProvider("https://ctz.solidwallet.io", 3))
# NID = 1

# Euljiro
# icon_service = IconService(HTTPProvider("https://test-ctz.solidwallet.io",3))
# NID = 2

# Yeouido
icon_service = IconService(HTTPProvider("https://bicon.net.solidwallet.io", 3))
NID = 3

# Pagoda
# icon_service = IconService(HTTPProvider("https://zicon.net.solidwallet.io", 3))
# NID = 80

# Custom
# icon_service = IconService(HTTPProvider("http://127.0.0.1:9000/", 3))

# Load Wallet using Keystore file

In [5]:
# wallet = KeyWallet.load("../keystore_ibriz.json","p@ssword1")
wallet = KeyWallet.load("./keystore/test1","1@MreddeviL")
deployer_address = wallet.get_address()
deployer_address

'hx177b10efe3018961a405cc7c20ee811c552831a5'

# Load Wallet Using Private Key

In [ ]:
private="---PrivateKey-------"
prepwallet = KeyWallet.load(bytes.fromhex(private))
prep_address = wallet.get_address()
prep_address

# Check Wallet Balance

In [ ]:
icon_service.get_balance(deployer_address)

# Contracts

In [6]:
#Load a pkl file if existed otherwise go to next step
contracts_file_custom = 'contracts_20201203030216.pkl'

with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)
    
contracts

{'CPFTreasury': {'zip': '/home/reddevilsdp/GitRepo/CPS/CPFTreasury',
  'SCORE': 'cxb176747630ffe4a22da7b2c713d7fbf063d19595'},
 'cps_score': {'zip': '/home/reddevilsdp/GitRepo/CPS/cps_score',
  'SCORE': 'cxa4b044b17158b964ed529a72d024eea80ac46944'},
 'CPSTreasury': {'zip': '/home/reddevilsdp/GitRepo/CPS/CPSTreasury',
  'SCORE': 'cx5887470bfad4434e7aadfb1397609a770371b9ed'}}

In [ ]:
# Add the contracts. Copy the output from this cell the the one above to save the contract addresses.
# Run this before deploying scores 
contracts=AutoVivification()

# directory names
deploy = ['cps_score', 'CPSTreasury', 'CPFTreasury']
# for directory in {"score"}:
directory=os.getcwd()
with os.scandir(directory) as it:
    for file in it:
        archive_name = directory + "/" + file.name
        if file.is_dir() and file.name in deploy:
            contracts[file.name]['zip'] = archive_name 
contracts

# Deploying Contract

In [ ]:
#this cell needs to run only when you need to deploy new set of contracts
for score, loc in contracts.items():
    params = {}
    print(score,loc)

    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(loc["zip"]))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[score]["SCORE"] = res.get('scoreAddress', '')
    
now = datetime.utcnow()
filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)
contracts

# Updating a Contract

In [50]:
# Contract Name
# 'cps_score', 'CPSTreasury', 'CPFTreasury'
contract = 'cps_score'

# Generates an instance of transaction for deploying score
transaction = DeployTransactionBuilder() \
            .from_(wallet.get_address()) \
            .to(contracts[contract]["SCORE"]) \
            .nid(NID) \
            .nonce(NONCE) \
            .content_type("application/zip") \
            .content(gen_deploy_data_content(contracts[contract]["zip"])) \
            .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(transaction, wallet,step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

0x7e6f5a6bb48cfbc1ccd01c34f36791a269721487dcb56b741f55ed0f4e844775


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Invalid params txHash'}


{'code': -32602, 'message': 'Invalid params txHash'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x7e6f5a6bb48cfbc1ccd01c34f36791a269721487dcb56b741f55ed0f4e844775',
 'blockHeight': 10308142,
 'blockHash': '0x7f5f22628ae8a17c722a6b73f2e41181e81a5a152b0b637e124eefe282700ce1',
 'txIndex': 2,
 'to': 'cxa4b044b17158b964ed529a72d024eea80ac46944',
 'scoreAddress': 'cxa4b044b17158b964ed529a72d024eea80ac46944',
 'stepUsed': 2125300000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 2125514760,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# After Deploying Setups

## Add Admins Address (only Owner)

In [ ]:
# Add a wallet address 
params = {"_address":"hx2176d13280d465b68afbf7f9be758e966f654629"}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .method("add_admin")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

### Remove Admin from Admin List

In [ ]:
#Remove the given wallet address from the admin list
params = {"_address":"hx2176d13280d465b68afbf7f9be758e966f654629"}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .method("remove_admin")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

## Set CPS Treasury SCORE

In [ ]:
# Set CPS Treasury SCORE Address on CPS_SCORE and CPF Treasury

contracts_name = ['cps_score','CPFTreasury']

params = {"_score":contracts["CPSTreasury"]['SCORE']}

for contract in contracts_name:
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[contract]['SCORE'])\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(NONCE)\
        .method("set_cps_treasury_score")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(tx_hash)

## Set CPF Treasury Score

In [ ]:
# Set CPF Treasury SCORE Address on CPS_SCORE and CPS Treasury

contracts_name = ['cps_score','CPSTreasury']

params = {"_score":contracts["CPFTreasury"]['SCORE']}

for contract in contracts_name:
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[contract]['SCORE'])\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(NONCE)\
        .method("set_cpf_treasury_score")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(tx_hash)

## Set CPS Score

In [ ]:
# Set CPS_SCORE SCORE Address on CPF TREASURY and CPS Treasury

contracts_name = ['CPFTreasury','CPSTreasury']

params = {"_score":contracts["cps_score"]['SCORE']}

for contract in contracts_name:
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[contract]['SCORE'])\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(NONCE)\
        .method("set_cps_score")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    print(tx_hash)

## Add Fund to CPF Treasury Score

In [47]:
# Add some fund to the CPF Treasury Score (Maximum 1M ICX)

params = {}

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["CPFTreasury"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .value(500*10**18)\
    .method("add_fund")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

0x7960c81742336be11283d9802e37d913ba9f0bdc4b4d9b9d78f4a997152fd266


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Invalid params txHash'}


{'code': -32602, 'message': 'Invalid params txHash'} (Json rpc error), Retrying in 4 seconds...


KeyboardInterrupt: 

## Set Penalty Amounts

In [ ]:
# Penalty Amount : Amount for Re-registering from the denylist.
# set the penalty amount for the P-Reps who are on Denylist 
# 3 Tiers penalty system

# params = {"_penalty":['1000','1500','2500']}
params = {"_penalty":['10','20','30']}

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .method("set_prep_penalty_amount")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

## Set Initial BlockHeight

In [32]:
# set a blockheight when the CPS to start from initial (only once)
params = {"_block_height":"0x396abb"}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .method("set_initialBlock")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

0xc9628f3ceffe324ac8822f804d71ef1fca7df3bbcdda61158801fd345ad682b6


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xc9628f3ceffe324ac8822f804d71ef1fca7df3bbcdda61158801fd345ad682b6',
 'blockHeight': 10307163,
 'blockHash': '0x874570d2a597f166f54cff369b1d8db446f51b6fe6e7c9987e9f8cdc769c8453',
 'txIndex': 9,
 'to': 'cxa4b044b17158b964ed529a72d024eea80ac46944',
 'stepUsed': 177800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1775400,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

# 

## Main Methods

### Register P-Rep

In [ ]:
params = {}

transaction = CallTransactionBuilder()\
    .from_(prepwallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("register_prep")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, prepwallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

### Listing P-Reps Address

In [30]:
## List the active registered prep with their details from System Score Address
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("get_PReps")\
                    .params({})\
                    .build()

result = icon_service.call(call)
result

[{'name': 'Testing P-Rep1',
  'address': 'hxf5bdb1625e4b7fc6de7d85f9dd921090b27ec7d0',
  'delegated': '0x572c417290705416d'},
 {'name': 'Testing P-Rep4',
  'address': 'hxef35c447e3f657315a6990da0ddaf4c61fa146e4',
  'delegated': '0x572c417290705416d'},
 {'name': 'Testing P-Rep8',
  'address': 'hxfd114a60eefa8e2c3de2d00dc5e41b1a0c7e8931',
  'delegated': '0x572c417290705416d'},
 {'name': 'Testing P-Rep1',
  'address': 'hx265305fb7d45e1edfba223503284d2b770605d4a',
  'delegated': '0x2bce06812557d416d'},
 {'name': 'Testing P-Rep5',
  'address': 'hxb4e90a285a79687ec148c29faabe6f71afa8a066',
  'delegated': '0x2bce06812557d416d'},
 {'name': 'Testing P-Rep6',
  'address': 'hxc60380ef4c1e76595a30fa40d7b519fb3c832db0',
  'delegated': '0x374a7f77c7abc3fb6816d'},
 {'name': 'Testing P-Rep7',
  'address': 'hx23823847f593ecb65c9e1ea81a789b02766280e8',
  'delegated': '0x2bce06812557d416d'}]

### Submit a Proposal

In [ ]:
## Submit a proposal 
### params : _project_title(str) : Project title for the proposal
### total_budget(int) : Total Budget for the project (ICX amount)
### sponsor_address(Address) : Address of the Sponsor P-Rep (must be a registered P-Rep)
### ipfs_hash(str) : IPFS Hash key for the proposal submission
### ipfs_link(str) : IPFS Link for the submitted proposal (Can see the details filled up and submitted)
### project_duration(int) : Total project duration (months) (Maximum 6)

params ={"_proposals": {
			"project_title": "ICON Mobile Wallet",
			"total_budget": "100",
			"sponsor_address": "hxe59539154a3d77023943c3bbd5fc3a081d697e6a",
			"ipfs_hash": "bafybeid3ucflkuettyzvzbsnqhvw5mtqkgbtes4s5yykhr6vd4ma7f7aiy",
			"ipfs_link": "https://gateway.ipfs.io/ipfs/bafybeid3ucflkuettyzvzbsnqhvw5mtqkgbtes4s5yykhr6vd4ma7f7aiy",
			"project_duration": "4"
		}}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .value(50000000000000000000)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("submit_proposal")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

### Vote Proposal

In [ ]:
### Vote a Proposal (Voting Period)

params = {"_vote": "_approve/_reject/_abstain",
          "_vote_reason": "Vote Reason",
          "_ipfs_key": "Proposal IPFS Hash"}
transaction = CallTransactionBuilder()\
    .from_(sdpwallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("vote_proposal")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, sdpwallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

### Submit Progress Report

In [ ]:
## Submit a Progress Report 
### params : progress_report_title(str) : Progress Report title for the progress report
### ipfs_hash(str) : IPFS Hash key for the main proposal
### report_hash(str): IPFS Hash key for the progress report submission
### ipfs_link(str) : IPFS Link for the submitted progress report (Can see the details filled up and submitted)
### percentage_completed(int) : Approx. project completion percentage
### budget_adjustment(bool) : True if progress report has budget adjustment application to increase the budget for the project
### additional_budget(int) (ICX amount) : application for additional budget proposed
### additional_month(int) : Additional month required for the additional work proposed

params = {"_progress_report": {
                "report_hash": "Progress Report IPFS HASH Key",
                "ipfs_hash": "Proposal IPFS HASH key",
                "progress_report_title": "Progress Report Title",
                "budget_adjustment": "0/1",
                "additional_budget": "0",
                "ipfs_link": "IPFS Submission Link",
                "percentage_completed": "Current Project Completion Status(int)",
                "additional_month": "0"
            }
          }

transaction = CallTransactionBuilder()\
    .from_(prep_wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("submit_progress_report")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, prep_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

### Vote Progress Report

In [ ]:
### Vote a progress Report (Voting Period)
params = {"_vote": "_approve/_reject",
          "_vote_reason": "Reason For the Vote",
          "_ipfs_key": "Proposal IPFS HASH",
          "_report_key":"Progress Report IPFS Hash",
          "_budget_adjustment_vote":"_approve/_reject"}

transaction = CallTransactionBuilder()\
    .from_(sdpwallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("vote_progress_report")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, sdpwallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

### Update a period

In [10]:
# To update a period from application period to voting period and vice-versa 
# after the block count of a period is completed
params = {}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["cps_score"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .method("update_period")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

0x4469d88537bdcb3f71cfdaea6d39df083387f920c7a22000416d6d0cd5e50795


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x4469d88537bdcb3f71cfdaea6d39df083387f920c7a22000416d6d0cd5e50795',
 'blockHeight': 10288134,
 'blockHash': '0x8609ecc4f918752c383ec4d31099106c35867199e44236532dfb7cbb9bc0d3cb',
 'txIndex': 1,
 'to': 'cxa4b044b17158b964ed529a72d024eea80ac46944',
 'stepUsed': 454500,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 454500,
 'eventLogs': [{'scoreAddress': 'cx5887470bfad4434e7aadfb1397609a770371b9ed',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx5887470bfad4434e7aadfb1397609a770371b9ed',
    'hxeee0810ddcacba64f26dc288f002861527fa7636',
    '0x15af1d78b58c40000'],
   'data': []},
  {'scoreAddress': 'cx5887470bfad4434e7aadfb1397609a770371b9ed',
   'indexed': ['ProposalFundSent(Address,int,str)',
    'hxeee0810ddcacba64f26dc288f002861527fa7636',
    '0x15af1d78b58c40000',
    'Installment No. 2 transferred to contributor'],
   'data': []},
  {'scoreAddress': 'cx5887470bfad4434e7aadfb1397609a770371b9ed',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx588

## Read Only Methods

In [ ]:
### check the wallet address status, also if they are on denylist the penalty amount to pay is also listed
params = {"_address":"hxb4e90a285a79687ec148c29faabe6f71afa8a066"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("login_prep")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

In [33]:
## Get period status with currentBlock, NextBlock to change the period, remainingTime for the period, 
## Currently running period name and the total period span

params = {}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("get_period_status")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

{'current_block': '0x9d4677',
 'next_block': '0x9d49df',
 'remaining_time': '0x6d0',
 'period_name': 'Application Period',
 'period_span': '0x708'}

In [31]:
### Get the list of Proposals with their details of provided status maximum 50 entries at once, 20 by default

## "_status" :["_sponsor_pending","_pending","_active","_paused","_completed","_rejected","_disqualifed"]
params = {"_status":"_paused"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("get_proposal_details")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

{'data': [{'ipfs_hash': 'bafybeiehmefbubn5nn2qqxej7mnlg5jtgoc2arjsltniksdrl23l575oyi',
   'project_title': 'World Changing Test Project',
   'timestamp': '0x5b61e7a4d1b60',
   'total_budget': '0x28',
   'project_duration': '0x3',
   'approved_reports': '0x2',
   'sponsor_address': 'hxb4e90a285a79687ec148c29faabe6f71afa8a066',
   'contributor_address': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
   'status': '_paused',
   'tx_hash': '5aaeb68d3ea8a62dcfc00903726b2bce2b2d601857847e253eab0355116d6b2b',
   'percentage_completed': '0x50',
   'sponsor_deposit_amount': '0x3782dace9d900000',
   'sponsored_timestamp': '0x5b6142d155033',
   'sponsor_deposit_status': 'bond_approved',
   'total_votes': '0x1b4a147c50ce59e5fa',
   'approved_votes': '0x1b4a147c50ce59e5fa',
   'rejected_votes': '0x0',
   'total_voters': '0x7',
   'approve_voters': '0x7',
   'reject_voters': '0x0'}],
 'count': '0x1'}

In [16]:
### Get the list of progress report with their details of provided status maximum 50 entries at once, 20 by default

## "_status" :["_waiting","_approved","_rejected"]
params = {"_status":"_approved"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("get_progress_reports")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

{'data': [{'ipfs_hash': 'bafybeifrv2vxo7utazawsxw5aj2yw6w3gvp5cu26oa4tzy37l2az52v3pm',
   'report_hash': 'bafybeidvbhbdw37xbc263d766kslymeshdfa456f3rs4e63zxv6tfsmxca',
   'progress_report_title': 'Progress Report II',
   'timestamp': '0x5b61389ebe023',
   'additional_budget': '0x0',
   'additional_month': '0x0',
   'status': '_approved',
   'tx_hash': '035b7a85182c1484b0e64c537129af78747c09221a4e8e38cb25067bfc23b2b2',
   'budget_adjustment': '0x1',
   'total_votes': '0x188d73234efcf50e44',
   'approved_votes': '0x188d73234efcf50e44',
   'rejected_votes': '0x0',
   'total_voters': '0x6',
   'approve_voters': '0x6',
   'reject_voters': '0x0',
   'budget_approved_votes': '0x0',
   'budget_rejected_votes': '0x0',
   'budget_approve_voters': '0x0',
   'budget_reject_voters': '0x0',
   'budget_adjustment_status': '',
   'project_title': 'Proposal 1',
   'contributor_address': 'hx2176d13280d465b68afbf7f9be758e966f654629'},
  {'ipfs_hash': 'bafybeifrv2vxo7utazawsxw5aj2yw6w3gvp5cu26oa4tzy37l2az

In [18]:
## List the progress report for the provided Proposal Ipfs_key (project key)
params = {"_ipfs_key":"bafybeifrv2vxo7utazawsxw5aj2yw6w3gvp5cu26oa4tzy37l2az52v3pm"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("get_progress_reports_by_proposal")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

{'data': [{'ipfs_hash': 'bafybeifrv2vxo7utazawsxw5aj2yw6w3gvp5cu26oa4tzy37l2az52v3pm',
   'report_hash': 'bafybeiddhf44nnyle476vfxsuyuply5kjhgvcbyveuwpx6jsv4vmqtz6mm',
   'progress_report_title': 'Progress Report I',
   'timestamp': '0x5b60e65353da5',
   'additional_budget': '0x0',
   'additional_month': '0x0',
   'status': '_progress_report_rejected',
   'tx_hash': '2c5fddbe02800909d985e78976b072a0177e540aefad748b85189d0b2f485451',
   'budget_adjustment': '0x1',
   'total_votes': '0x10682a8439a4977924',
   'approved_votes': '0x10682a8439a4977924',
   'rejected_votes': '0x0',
   'total_voters': '0x4',
   'approve_voters': '0x4',
   'reject_voters': '0x0',
   'budget_approved_votes': '0x0',
   'budget_rejected_votes': '0x0',
   'budget_approve_voters': '0x0',
   'budget_reject_voters': '0x0',
   'budget_adjustment_status': '',
   'project_title': 'Proposal 1',
   'contributor_address': 'hx2176d13280d465b68afbf7f9be758e966f654629'},
  {'ipfs_hash': 'bafybeifrv2vxo7utazawsxw5aj2yw6w3gvp5c

In [37]:
## To check if the provided contributor has any progress report to submit in the current Application Period
params = {"_wallet_address":"hx2176d13280d465b68afbf7f9be758e966f654629"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("get_active_proposals")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

[{'project_title': 'Proposal 1',
  'ipfs_hash': 'bafybeifrv2vxo7utazawsxw5aj2yw6w3gvp5cu26oa4tzy37l2az52v3pm',
  'new_progress_report': '0x1',
  'last_progress_report': '0x1'}]

In [13]:
## Get project amount and total amount pending for the given contributor address

params = {"_wallet_address":"hxeee0810ddcacba64f26dc288f002861527fa7636"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["CPSTreasury"]['SCORE'])\
                    .method("get_contributor_projected_fund")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

{'data': [], 'project_count': '0x0', 'total_amount': '0x0'}

In [15]:
## Get project amount and total amount pending for the given sponsorAddress 

params = {"_wallet_address":"hxef35c447e3f657315a6990da0ddaf4c61fa146e4"}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["CPSTreasury"]['SCORE'])\
                    .method("get_sponsor_projected_fund")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

{'data': [],
 'project_count': '0x0',
 'total_amount': '0x0',
 'total_sponsor_bond': '0x0'}

In [14]:
## Get project amount and total amount pending for the given sponsorAddress 

params = {}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["cps_score"]['SCORE'])\
                    .method("get_sponsors_record")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

{'hxf5bdb1625e4b7fc6de7d85f9dd921090b27ec7d0': '0x1',
 'hxef35c447e3f657315a6990da0ddaf4c61fa146e4': '0x1',
 'hxb4e90a285a79687ec148c29faabe6f71afa8a066': '0x1'}

In [42]:
## Get project amount and total amount pending for the given sponsorAddress 

params = {}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts["CPFTreasury"]['SCORE'])\
                    .method("get_total_fund")\
                    .params(params)\
                    .build()

result = icon_service.call(call)
result

'0x0'

In [46]:
# To update a period from application period to voting period and vice-versa 
# after the block count of a period is completed
params = {"_value":1000000}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts["CPFTreasury"]['SCORE'])\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(NONCE)\
    .method("set_maximum_treasury_fund")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
print(tx_hash)

get_tx_result(tx_hash)

0x9526d23248a5c29381776112ac00ba12fc62a47e9dd8b8f3c2d5b0169d79c4b8


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x9526d23248a5c29381776112ac00ba12fc62a47e9dd8b8f3c2d5b0169d79c4b8',
 'blockHeight': 10307477,
 'blockHash': '0xf4bb10fb20c6e3a61287ad10f76b160dcabac391533ce33375936dbf2130d57d',
 'txIndex': 1,
 'to': 'cxb176747630ffe4a22da7b2c713d7fbf063d19595',
 'stepUsed': 142120,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 142120,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00